In [17]:
"""
Single-Prompt LLM Analysis
Analyzes one specific prompt folder with multiple LLMs (fast & parallel-friendly)
"""

import json
import requests
from pathlib import Path
import base64
import time
from PIL import Image
import io

# ===== AWS BEDROCK CONFIGURATION =====
API_ENDPOINT = "https://ctwa92wg1b.execute-api.us-east-1.amazonaws.com/prod/invoke"
TEAM_ID = "team_the_great_hack_2025_022"
API_TOKEN = "znqXT5zCmCynAx-kyx_hldrxvSeyaWvxzx55vB5mfNg"

# ===== EXPANDED LLM SELECTION (All Claude models) =====
LLMS_TO_TEST = {
    # Claude 3 Series
    "claude_3_opus": "us.anthropic.claude-3-opus-20240229-v1:0",  # Most Powerful (3.x)
    "claude_3_sonnet": "us.anthropic.claude-3-sonnet-20240229-v1:0",
    "claude_3_haiku": "us.anthropic.claude-3-haiku-20240307-v1:0",  # Fastest (3.x)
    
    # Claude 3.5 Series
    "claude_3.5_sonnet": "us.anthropic.claude-3-5-sonnet-20241022-v2:0",  # Recommended
    "claude_3.5_haiku": "us.anthropic.claude-3-5-haiku-20241022-v1:0",  # Fast
    
    # Claude 4 Series (Latest)
    "claude_4_opus": "us.anthropic.claude-opus-4-20250514-v1:0",
    "claude_4_sonnet": "us.anthropic.claude-sonnet-4-20250514-v1:0",
    "claude_4.5_sonnet": "us.anthropic.claude-sonnet-4-5-20250929-v1:0",  # Latest & Smartest
    "claude_4.5_haiku": "us.anthropic.claude-haiku-4-5-20251001-v1:0",
}

# Model-specific configurations
MODEL_CONFIGS = {
    # All Claude models support temperature and vision
    "no_temperature": [],
    "small_context": [],
    "limited_vision": []
}

# ===== HELPER FUNCTIONS =====
def resize_image_if_needed(image_path, max_dimension=1568, quality=85):
    """
    Resize image to fit within API limits while maintaining quality
    
    Args:
        image_path: Path to image file
        max_dimension: Max width or height (Claude supports up to 1568px)
        quality: JPEG quality (85 is good balance)
    
    Returns:
        Base64 encoded image string
    """
    try:
        with Image.open(image_path) as img:
            # Convert to RGB if necessary (handles RGBA, grayscale, etc.)
            if img.mode not in ('RGB', 'L'):
                img = img.convert('RGB')
            
            # Get original dimensions
            width, height = img.size
            
            # Calculate if resize is needed
            if max(width, height) > max_dimension:
                # Calculate new dimensions maintaining aspect ratio
                if width > height:
                    new_width = max_dimension
                    new_height = int(height * (max_dimension / width))
                else:
                    new_height = max_dimension
                    new_width = int(width * (max_dimension / height))
                
                # Resize with high-quality resampling
                img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
            
            # Save to bytes buffer as JPEG (much smaller than PNG)
            buffer = io.BytesIO()
            img.save(buffer, format='JPEG', quality=quality, optimize=True)
            buffer.seek(0)
            
            # Encode to base64
            img_b64 = base64.b64encode(buffer.read()).decode('utf-8')
            
            return img_b64, 'image/jpeg'
    
    except Exception as e:
        raise Exception(f"Failed to process image {image_path}: {str(e)}")

def encode_image_to_base64(image_path):
    """Convert image to base64 for API (legacy, use resize_image_if_needed instead)"""
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def call_bedrock_llm(model_id, prompt, images=None, debug=False):
    """Call AWS Bedrock via API"""
    headers = {
        "Content-Type": "application/json",
        "x-api-key": API_TOKEN
    }
    
    content = []
    
    # Determine image size based on model capabilities
    if model_id in MODEL_CONFIGS["small_context"]:
        max_dim = 800  # Smaller images for limited context models
        quality = 70
    else:
        max_dim = 1200  # Standard size
        quality = 80
    
    # Check if model has limited vision support
    if model_id in MODEL_CONFIGS["limited_vision"]:
        if debug:
            print(f"\n      ⚠️  Model has limited vision support, using only first 2 images")
        images = images[:2] if images else []  # Only use first 2 images
    
    if images:
        for img_path in images:
            try:
                # Use optimized image encoding
                img_b64, media_type = resize_image_if_needed(img_path, max_dimension=max_dim, quality=quality)
                
                if debug:
                    print(f"\n      📎 Encoded image: {img_path} ({len(img_b64)} chars)")
                
                content.append({
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": media_type,
                        "data": img_b64
                    }
                })
            except Exception as e:
                if debug:
                    print(f"\n      ⚠️  Failed to encode {img_path}: {e}")
                return f"ERROR: Image encoding failed for {img_path}: {str(e)}"
    
    content.append({"type": "text", "text": prompt})
    
    # Build payload with model-specific parameters
    payload = {
        "team_id": TEAM_ID,
        "api_token": API_TOKEN,
        "model": model_id,
        "messages": [{"role": "user", "content": content}],
        "max_tokens": 2000,
    }
    
    # Only add temperature if model supports it
    if model_id not in MODEL_CONFIGS["no_temperature"]:
        payload["temperature"] = 0.3
    
    if debug:
        print(f"\n      📤 Payload size: {len(json.dumps(payload))} bytes")
        print(f"      📤 Content blocks: {len(content)} ({len([c for c in content if c['type']=='image'])} images, {len([c for c in content if c['type']=='text'])} text)")
        print(f"      📤 Temperature: {'included' if 'temperature' in payload else 'excluded (not supported)'}")
    
    try:
        response = requests.post(API_ENDPOINT, headers=headers, json=payload, timeout=90)
        
        if debug:
            print(f"\n      📊 Status: {response.status_code}")
            print(f"      📊 Response size: {len(response.text)} bytes")
        
        # Check status before parsing JSON
        if response.status_code != 200:
            error_text = response.text[:500]
            if debug:
                print(f"\n      ❌ Error response: {error_text}")
            return f"ERROR: HTTP {response.status_code} - {error_text}"
        
        result = response.json()
        
        if debug:
            print(f"      📊 Response keys: {list(result.keys())}")
        
        if "content" in result and len(result["content"]) > 0:
            return result["content"][0]["text"]
        else:
            fallback = result.get("completion", "No response")
            if debug:
                print(f"      ⚠️  Using fallback: {fallback[:100]}")
            return fallback
    
    except requests.exceptions.Timeout:
        return f"ERROR: Request timeout (>90s)"
    
    except requests.exceptions.RequestException as e:
        return f"ERROR: Request failed - {type(e).__name__}: {str(e)}"
    
    except json.JSONDecodeError as e:
        return f"ERROR: Invalid JSON response - {str(e)}"
    
    except Exception as e:
        if debug:
            import traceback
            print(f"\n      ❌ Exception traceback:")
            traceback.print_exc()
        return f"ERROR: {type(e).__name__}: {str(e)}"

def generate_analysis_prompt(prompt_text, scenario_hint=""):
    """Create analysis prompt for LLM"""
    return f"""You are an expert in AI-generated image quality assessment and prompt engineering for diffusion models.

**Task**: Analyze this logo transformation for Christmas merchandise design.

**Prompt used**: "{prompt_text}"
**Design goal**: {scenario_hint if scenario_hint else "Holiday merchandise design"}
**Model**: FLUX.1-Kontext-dev (FP16)

**Images provided**:
1. Input logo (original)
2. Generated output (transformed design)
3. Word attribution (3 rows: WITH word, WITHOUT word, difference heatmap)
4. Evolution grid (diffusion timesteps)
5. Timestep attention evolution

**Provide concise analysis (2-3 sentences each)**:

1. **Prompt Adherence**: Did the model follow instructions? What elements match the prompt?

2. **Logo Preservation**: Is the original logo recognizable? Any brand elements lost?

3. **Design Suitability**: Would this work for merchandise? Consider printing/manufacturing.

4. **Creative Execution**: How well were "Christmas"/"festive" elements interpreted? Tasteful?

5. **Technical Quality**: Any artifacts, distortions, or quality issues?

6. **Word Attribution Insights**: Which words (from row 2 of attribution image) had most impact? Any redundant?

7. **Prompt Improvements**: 2-3 specific changes to improve output (reference word attribution).

Be direct and actionable."""

def analyze_single_prompt_folder(prompt_folder_path, scenario_name=""):
    """
    Analyze a single prompt folder with all LLMs
    
    Args:
        prompt_folder_path: Path to folder like "flux_experiments/run_XXX/tshirt_design/prompt_0_base"
        scenario_name: Optional hint like "t-shirt design" (auto-detected from path if empty)
    """
    
    prompt_path = Path(prompt_folder_path)
    
    if not prompt_path.exists():
        print(f"❌ Folder not found: {prompt_path}")
        return
    
    # Auto-detect scenario from path
    if not scenario_name:
        parts = prompt_path.parts
        if len(parts) >= 2:
            scenario_name = parts[-2].replace('_', ' ')
    
    # Load metadata to get prompt text
    metadata_file = prompt_path / "metadata.json"
    if metadata_file.exists():
        with open(metadata_file) as f:
            metadata = json.load(f)
        prompt_text = metadata.get("prompt", "Unknown prompt")
    else:
        prompt_text = "Prompt not found in metadata"
    
    print(f"\n{'='*70}")
    print(f"📁 ANALYZING: {prompt_path.name}")
    print(f"📝 Prompt: {prompt_text[:80]}...")
    print(f"🎨 Scenario: {scenario_name}")
    print(f"{'='*70}\n")
    
    # Collect images
    images_to_analyze = []
    
    image_files = [
        "input_image.png",
        "final_output.png",
        "word_attribution_complete.png",
        "evolution_grid.png"
    ]
    
    for img_name in image_files:
        img_path = prompt_path / img_name
        if img_path.exists():
            images_to_analyze.append(str(img_path))
    
    # Add timestep evolution if exists
    timestep_imgs = list(prompt_path.glob("timestep_evolution_*.png"))
    if timestep_imgs:
        images_to_analyze.append(str(timestep_imgs[0]))
    
    print(f"📸 Found {len(images_to_analyze)} images to analyze\n")
    
    # Generate analysis prompt
    analysis_prompt = generate_analysis_prompt(prompt_text, scenario_name)
    
    # Create output directory
    output_dir = prompt_path / "llm_analysis"
    output_dir.mkdir(exist_ok=True)
    
    # Run each LLM
    results = {}
    
    # DEBUG MODE: Enable detailed logging for ALL models initially
    debug_mode = True
    success_count = 0
    
    for llm_name, llm_model_id in LLMS_TO_TEST.items():
        print(f"🤖 {llm_name:20s} ", end='', flush=True)
        
        start_time = time.time()
        response = call_bedrock_llm(llm_model_id, analysis_prompt, images_to_analyze, debug=debug_mode)
        duration = time.time() - start_time
        
        if response.startswith("ERROR"):
            print(f"❌ Failed ({duration:.1f}s)")
            # Print error details
            if debug_mode:
                print(f"   🔍 ERROR: {response[:300]}\n")
        else:
            print(f"✅ Done ({duration:.1f}s, {len(response)} chars)")
            success_count += 1
            # Disable debug after first success to reduce spam
            if success_count >= 1:
                debug_mode = False
        
        # Save result
        result = {
            "llm": llm_name,
            "model_id": llm_model_id,
            "prompt": prompt_text,
            "scenario": scenario_name,
            "analysis": response,
            "response_time_seconds": duration,
            "response_length": len(response),
            "images_analyzed": len(images_to_analyze),
            "timestamp": time.time()
        }
        
        results[llm_name] = result
        
        # Save individual JSON
        with open(output_dir / f"{llm_name}_analysis.json", "w") as f:
            json.dump(result, f, indent=2)
        
        time.sleep(0.5)  # Brief rate limit
    
    # Create summary markdown
    create_summary_report(results, output_dir, prompt_text)
    
    print(f"\n{'='*70}")
    print(f"✅ COMPLETE! Analyzed with {len(results)} LLMs")
    print(f"📊 Results saved to: {output_dir}")
    print(f"📄 Report: {output_dir}/ANALYSIS_SUMMARY.md")
    print(f"{'='*70}\n")
    
    return results

def create_summary_report(results, output_dir, prompt_text):
    """Create markdown summary of all LLM analyses"""
    
    report_path = output_dir / "ANALYSIS_SUMMARY.md"
    
    with open(report_path, "w") as f:
        f.write(f"# LLM Analysis Summary\n\n")
        f.write(f"**Prompt**: {prompt_text}\n\n")
        f.write(f"**Generated**: {time.strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        f.write("---\n\n")
        
        # Performance comparison table
        f.write("## ⚡ Performance Comparison\n\n")
        f.write("| LLM | Response Time | Length | Status |\n")
        f.write("|-----|---------------|--------|--------|\n")
        
        for llm_name, result in results.items():
            status = "✅ Success" if not result['analysis'].startswith("ERROR") else "❌ Failed"
            f.write(f"| {llm_name} | {result['response_time_seconds']:.1f}s | {result['response_length']} chars | {status} |\n")
        
        f.write("\n---\n\n")
        
        # Individual analyses
        f.write("## 📝 Detailed Analyses\n\n")
        
        for llm_name, result in results.items():
            f.write(f"### 🤖 {llm_name.upper()}\n\n")
            f.write(f"*Model: {result['model_id']}*\n\n")
            f.write(f"{result['analysis']}\n\n")
            f.write("---\n\n")
    
    print(f"📄 Summary saved: {report_path}")

# ===== BATCH RUNNER (analyze multiple folders) =====
def analyze_multiple_prompts(prompt_folders):
    """
    Analyze multiple prompt folders in sequence
    
    Args:
        prompt_folders: List of paths to prompt folders
    """
    
    all_results = {}
    
    for folder_path in prompt_folders:
        results = analyze_single_prompt_folder(folder_path)
        all_results[str(folder_path)] = results
        print("\n" + "="*70 + "\n")
    
    print(f"🎉 Batch complete! Analyzed {len(all_results)} prompt folders")
    return all_results

# ===== USAGE EXAMPLES =====

# Example 1: Analyze single prompt folder
analyze_single_prompt_folder(
    "flux_experiments/run_20251116_013857/tshirt_design/prompt_0_base"
)

# Example 2: Analyze all prompts in a scenario
"""
scenario_path = Path("flux_experiments/run_20251116_013857/tshirt_design")
all_prompts = sorted(scenario_path.glob("prompt_*"))

for prompt_folder in all_prompts:
    analyze_single_prompt_folder(str(prompt_folder))
"""

# Example 3: Analyze specific prompts only
"""
analyze_multiple_prompts([
    "flux_experiments/run_20251116_013857/tshirt_design/prompt_0_base",
    "flux_experiments/run_20251116_013857/tshirt_design/prompt_2_variant2",
    "flux_experiments/run_20251116_013857/mug_design/prompt_1_variant1"
])
"""


📁 ANALYZING: prompt_0_base
📝 Prompt: adapt logo for holiday t-shirt print with seasonal elements...
🎨 Scenario: tshirt design

📸 Found 5 images to analyze

🤖 claude_3_opus        
      📎 Encoded image: flux_experiments/run_20251116_013857/tshirt_design/prompt_0_base/input_image.png (64296 chars)

      📎 Encoded image: flux_experiments/run_20251116_013857/tshirt_design/prompt_0_base/final_output.png (160708 chars)

      📎 Encoded image: flux_experiments/run_20251116_013857/tshirt_design/prompt_0_base/word_attribution_complete.png (276852 chars)

      📎 Encoded image: flux_experiments/run_20251116_013857/tshirt_design/prompt_0_base/evolution_grid.png (317276 chars)

      📎 Encoded image: flux_experiments/run_20251116_013857/tshirt_design/prompt_0_base/timestep_evolution_seasonal.png (145936 chars)

      📤 Payload size: 967118 bytes
      📤 Content blocks: 6 (5 images, 1 text)
      📤 Temperature: included

      📊 Status: 200
      📊 Response size: 2120 bytes
      📊 Response keys

'\nanalyze_multiple_prompts([\n    "flux_experiments/run_20251116_013857/tshirt_design/prompt_0_base",\n    "flux_experiments/run_20251116_013857/tshirt_design/prompt_2_variant2",\n    "flux_experiments/run_20251116_013857/mug_design/prompt_1_variant1"\n])\n'

In [23]:
"""
Comprehensive Single-Prompt LLM Analysis with Scoring
Uses ALL generated images and provides quantitative comparison metrics
"""

import json
import requests
from pathlib import Path
import base64
import time
from PIL import Image
import io
import pandas as pd

# ===== AWS BEDROCK CONFIGURATION =====
API_ENDPOINT = "https://ctwa92wg1b.execute-api.us-east-1.amazonaws.com/prod/invoke"
TEAM_ID = "team_the_great_hack_2025_022"
API_TOKEN = "znqXT5zCmCynAx-kyx_hldrxvSeyaWvxzx55vB5mfNg"

# ===== ALL CLAUDE MODELS =====
LLMS_TO_TEST = {
    "claude_3_opus": "us.anthropic.claude-3-opus-20240229-v1:0",
    "claude_3_sonnet": "us.anthropic.claude-3-sonnet-20240229-v1:0",
    "claude_3_haiku": "us.anthropic.claude-3-haiku-20240307-v1:0",
    "claude_3.5_sonnet": "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    "claude_3.5_haiku": "us.anthropic.claude-3-5-haiku-20241022-v1:0",
    "claude_4_opus": "us.anthropic.claude-opus-4-20250514-v1:0",
    "claude_4_sonnet": "us.anthropic.claude-sonnet-4-20250514-v1:0",
    "claude_4.5_sonnet": "us.anthropic.claude-sonnet-4-5-20250929-v1:0",
    "claude_4.5_haiku": "us.anthropic.claude-haiku-4-5-20251001-v1:0",
}

# ===== IMAGE PROCESSING =====
def resize_image_if_needed(image_path, max_dimension=1200, quality=80):
    """Resize and optimize image for API"""
    try:
        with Image.open(image_path) as img:
            if img.mode not in ('RGB', 'L'):
                img = img.convert('RGB')
            
            width, height = img.size
            
            if max(width, height) > max_dimension:
                if width > height:
                    new_width = max_dimension
                    new_height = int(height * (max_dimension / width))
                else:
                    new_height = max_dimension
                    new_width = int(width * (max_dimension / height))
                
                img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
            
            buffer = io.BytesIO()
            img.save(buffer, format='JPEG', quality=quality, optimize=True)
            buffer.seek(0)
            
            img_b64 = base64.b64encode(buffer.read()).decode('utf-8')
            return img_b64, 'image/jpeg'
    
    except Exception as e:
        raise Exception(f"Failed to process {image_path}: {str(e)}")

# ===== API CALL =====
def call_bedrock_llm(model_id, prompt, images=None):
    """Call AWS Bedrock via API"""
    headers = {
        "Content-Type": "application/json",
        "x-api-key": API_TOKEN
    }
    
    content = []
    
    if images:
        for img_path in images:
            try:
                img_b64, media_type = resize_image_if_needed(img_path, max_dimension=1200, quality=80)
                content.append({
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": media_type,
                        "data": img_b64
                    }
                })
            except Exception as e:
                return f"ERROR: Image encoding failed for {img_path}: {str(e)}"
    
    content.append({"type": "text", "text": prompt})
    
    payload = {
        "team_id": TEAM_ID,
        "api_token": API_TOKEN,
        "model": model_id,
        "messages": [{"role": "user", "content": content}],
        "max_tokens": 3000,  # Increased for comprehensive analysis
        "temperature": 0.3
    }
    
    try:
        response = requests.post(API_ENDPOINT, headers=headers, json=payload, timeout=120)
        
        if response.status_code != 200:
            return f"ERROR: HTTP {response.status_code} - {response.text[:500]}"
        
        result = response.json()
        
        if "content" in result and len(result["content"]) > 0:
            return result["content"][0]["text"]
        else:
            return result.get("completion", "No response")
    
    except Exception as e:
        return f"ERROR: {type(e).__name__}: {str(e)}"

# ===== COMPREHENSIVE ANALYSIS PROMPT =====
def generate_comprehensive_analysis_prompt(prompt_text, scenario_hint, image_count):
    """Create detailed analysis prompt that references ALL images"""
    return f"""You are an expert in AI-generated image quality assessment and prompt engineering for diffusion models.

**Task**: Provide a comprehensive analysis of this logo transformation for Christmas merchandise design.

**Prompt used**: "{prompt_text}"
**Design goal**: {scenario_hint}
**Model**: FLUX.1-Kontext-dev (FP16)

**Images provided ({image_count} total)**:
1. **Input logo** - Original design
2. **Final output** - Transformed result
3. **Word attribution (3-row visualization)** - Shows WITH/WITHOUT/HEATMAP for each key word
4. **Individual ablated outputs** - Separate images showing result when each word is removed
5. **Evolution grid** - Diffusion process across timesteps
6. **Timestep evolution** - How attention changes over denoising steps
7. **Snapshots** - Intermediate generation stages

**Provide detailed analysis**:

### 1. Prompt Adherence (3-4 sentences)
Examine the final output and compare to the prompt. Which specific elements were successfully implemented? What was missing or misinterpreted?

### 2. Logo Preservation (3-4 sentences)
Looking at input vs output, is the original logo recognizable? Which brand elements were maintained vs lost? Reference specific visual details.

### 3. Design Suitability for {scenario_hint} (3-4 sentences)
Would this work for actual merchandise? Consider printing, colors, scalability, and practical manufacturing constraints.

### 4. Creative Execution (3-4 sentences)
How well were Christmas/festive elements interpreted? Are they tasteful and appropriate? Reference specific design choices visible in the images.

### 5. Technical Quality (3-4 sentences)
Examine all images (especially evolution grid and snapshots) for artifacts, distortions, consistency issues, or quality problems.

### 6. Word Attribution Analysis (4-5 sentences)
Study the word attribution visualization AND individual ablated images. Which words had the strongest visual impact? Were any redundant? What spatial patterns do you see in the heatmaps? Reference specific words and their effects.

### 7. Evolution & Process Insights (3-4 sentences)
Looking at the evolution grid and timestep snapshots, how did the image develop? Were there any interesting intermediate stages? Did the process appear stable?

### 8. Prompt Improvement Recommendations (4-5 sentences)
Based on word attribution and ablation results, provide 3-5 specific prompt modifications that would improve the output. Be concrete and actionable.

### 9. Overall Quality Score
Provide a score out of 10 for:
- **Prompt Adherence**: X/10
- **Logo Preservation**: X/10
- **Design Suitability**: X/10
- **Creative Execution**: X/10
- **Technical Quality**: X/10
- **OVERALL SCORE**: X/10

**Be thorough, reference specific images, and provide actionable insights.**"""

# ===== COLLECT ALL IMAGES =====
def collect_all_images(prompt_path):
    """
    Collect ALL images from the prompt folder for comprehensive analysis
    
    Returns:
        List of image paths in priority order
    """
    images = []
    
    # Priority images (always include first)
    priority_files = [
        "input_image.png",
        "final_output.png",
        "word_attribution_complete.png"
    ]
    
    for img_name in priority_files:
        img_path = prompt_path / img_name
        if img_path.exists():
            images.append(str(img_path))
    
    # Individual ablated images (sorted alphabetically)
    ablated_imgs = sorted(prompt_path.glob("ablated_without_*.png"))
    for img in ablated_imgs:
        images.append(str(img))
    
    # Evolution grid
    evolution_img = prompt_path / "evolution_grid.png"
    if evolution_img.exists():
        images.append(str(evolution_img))
    
    # Timestep evolution
    timestep_imgs = sorted(prompt_path.glob("timestep_evolution_*.png"))
    for img in timestep_imgs:
        images.append(str(img))
    
    # Snapshots folder (if exists)
    snapshot_dir = prompt_path / "snapshots"
    if snapshot_dir.exists():
        snapshot_imgs = sorted(snapshot_dir.glob("*.png"))
        # Limit to first 5 snapshots to avoid overwhelming the model
        for img in snapshot_imgs[:5]:
            images.append(str(img))
    
    return images

# ===== EXTRACT SCORES =====
def extract_scores_from_analysis(analysis_text):
    """
    Extract numerical scores from LLM analysis
    
    Returns:
        Dict of scores or None if extraction fails
    """
    scores = {
        "prompt_adherence": None,
        "logo_preservation": None,
        "design_suitability": None,
        "creative_execution": None,
        "technical_quality": None,
        "overall": None
    }
    
    try:
        # Look for score patterns like "X/10" or "Score: X"
        import re
        
        # Define patterns for each category
        patterns = {
            "prompt_adherence": r"Prompt Adherence[:\s]*(\d+(?:\.\d+)?)\s*/\s*10",
            "logo_preservation": r"Logo Preservation[:\s]*(\d+(?:\.\d+)?)\s*/\s*10",
            "design_suitability": r"Design Suitability[:\s]*(\d+(?:\.\d+)?)\s*/\s*10",
            "creative_execution": r"Creative Execution[:\s]*(\d+(?:\.\d+)?)\s*/\s*10",
            "technical_quality": r"Technical Quality[:\s]*(\d+(?:\.\d+)?)\s*/\s*10",
            "overall": r"OVERALL SCORE[:\s]*(\d+(?:\.\d+)?)\s*/\s*10"
        }
        
        for category, pattern in patterns.items():
            match = re.search(pattern, analysis_text, re.IGNORECASE)
            if match:
                scores[category] = float(match.group(1))
        
        return scores
    
    except Exception as e:
        print(f"      ⚠️  Score extraction failed: {e}")
        return scores

# ===== MAIN ANALYSIS FUNCTION =====
def analyze_single_prompt_folder(prompt_folder_path, scenario_name=""):
    """
    Comprehensive analysis with ALL images and scoring
    """
    
    prompt_path = Path(prompt_folder_path)
    
    if not prompt_path.exists():
        print(f"❌ Folder not found: {prompt_path}")
        return
    
    # Auto-detect scenario
    if not scenario_name:
        parts = prompt_path.parts
        if len(parts) >= 2:
            scenario_name = parts[-2].replace('_', ' ')
    
    # Load metadata
    metadata_file = prompt_path / "metadata.json"
    if metadata_file.exists():
        with open(metadata_file) as f:
            metadata = json.load(f)
        prompt_text = metadata.get("prompt", "Unknown prompt")
    else:
        prompt_text = "Prompt not found"
    
    print(f"\n{'='*70}")
    print(f"📁 ANALYZING: {prompt_path.name}")
    print(f"📝 Prompt: {prompt_text[:80]}...")
    print(f"🎨 Scenario: {scenario_name}")
    print(f"{'='*70}\n")
    
    # Collect ALL images
    print("📸 Collecting all images...")
    images_to_analyze = collect_all_images(prompt_path)
    
    print(f"   ✅ Found {len(images_to_analyze)} images:")
    for i, img in enumerate(images_to_analyze, 1):
        img_name = Path(img).name
        print(f"      {i}. {img_name}")
    print()
    
    # Generate comprehensive analysis prompt
    analysis_prompt = generate_comprehensive_analysis_prompt(
        prompt_text, scenario_name, len(images_to_analyze)
    )
    
    # Create output directory
    output_dir = prompt_path / "llm_analysis"
    output_dir.mkdir(exist_ok=True)
    
    # Run each LLM
    results = {}
    all_scores = {}
    
    for llm_name, llm_model_id in LLMS_TO_TEST.items():
        print(f"🤖 {llm_name:20s} ", end='', flush=True)
        
        start_time = time.time()
        response = call_bedrock_llm(llm_model_id, analysis_prompt, images_to_analyze)
        duration = time.time() - start_time
        
        if response.startswith("ERROR"):
            print(f"❌ Failed ({duration:.1f}s)")
            scores = None
        else:
            print(f"✅ Done ({duration:.1f}s, {len(response)} chars)")
            # Extract scores
            scores = extract_scores_from_analysis(response)
            all_scores[llm_name] = scores
            if scores and scores['overall']:
                print(f"      📊 Overall Score: {scores['overall']}/10")
        
        # Save result
        result = {
            "llm": llm_name,
            "model_id": llm_model_id,
            "prompt": prompt_text,
            "scenario": scenario_name,
            "analysis": response,
            "scores": scores,
            "response_time_seconds": duration,
            "response_length": len(response),
            "images_analyzed": len(images_to_analyze),
            "timestamp": time.time()
        }
        
        results[llm_name] = result
        
        # Save individual JSON
        with open(output_dir / f"{llm_name}_analysis.json", "w") as f:
            json.dump(result, f, indent=2)
        
        time.sleep(0.5)
    
    # Create comprehensive reports
    create_comprehensive_report(results, all_scores, output_dir, prompt_text)
    create_score_comparison_matrix(all_scores, output_dir)
    
    print(f"\n{'='*70}")
    print(f"✅ COMPLETE! Analyzed with {len(results)} LLMs")
    print(f"📊 Results saved to: {output_dir}")
    print(f"📄 Reports: ANALYSIS_SUMMARY.md, SCORE_COMPARISON.md")
    print(f"{'='*70}\n")
    
    return results, all_scores

# ===== REPORTING =====
def create_comprehensive_report(results, all_scores, output_dir, prompt_text):
    """Create detailed markdown report"""
    
    report_path = output_dir / "ANALYSIS_SUMMARY.md"
    
    with open(report_path, "w") as f:
        f.write(f"# Comprehensive LLM Analysis Report\n\n")
        f.write(f"**Prompt**: {prompt_text}\n\n")
        f.write(f"**Generated**: {time.strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        f.write("---\n\n")
        
        # Performance table
        f.write("## ⚡ Performance & Scoring Overview\n\n")
        f.write("| LLM | Response Time | Length | Overall Score | Status |\n")
        f.write("|-----|---------------|--------|---------------|--------|\n")
        
        for llm_name, result in results.items():
            status = "✅" if not result['analysis'].startswith("ERROR") else "❌"
            score = result['scores']['overall'] if result['scores'] and result['scores']['overall'] else "N/A"
            f.write(f"| {llm_name} | {result['response_time_seconds']:.1f}s | {result['response_length']} chars | {score}/10 | {status} |\n")
        
        f.write("\n---\n\n")
        
        # Individual analyses
        f.write("## 📝 Detailed Analyses\n\n")
        
        for llm_name, result in results.items():
            f.write(f"### 🤖 {llm_name.upper()}\n\n")
            f.write(f"*Model: {result['model_id']}*\n\n")
            
            if result['scores']:
                f.write(f"**Scores**:\n")
                for category, score in result['scores'].items():
                    if score is not None:
                        f.write(f"- {category.replace('_', ' ').title()}: {score}/10\n")
                f.write("\n")
            
            f.write(f"**Analysis**:\n\n")
            f.write(f"{result['analysis']}\n\n")
            f.write("---\n\n")
    
    print(f"📄 Summary saved: {report_path}")

def create_score_comparison_matrix(all_scores, output_dir):
    """Create score comparison matrix across all LLMs (no tabulate dependency)"""
    
    report_path = output_dir / "SCORE_COMPARISON.md"
    
    # Build comparison data
    score_data = []
    for llm_name, scores in all_scores.items():
        if scores:
            row = {"LLM": llm_name}
            row.update(scores)
            score_data.append(row)
    
    if not score_data:
        print("⚠️  No scores to compare")
        return
    
    # Convert to simple dict structure for easy processing
    df_data = pd.DataFrame(score_data)
    
    with open(report_path, "w") as f:
        f.write("# LLM Score Comparison Matrix\n\n")
        f.write(f"**Generated**: {time.strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        f.write("---\n\n")
        
        # Overall ranking
        if 'overall' in df_data.columns:
            df_sorted = df_data.sort_values('overall', ascending=False)
            f.write("## 🏆 Overall Ranking\n\n")
            f.write("| Rank | LLM | Overall Score |\n")
            f.write("|------|-----|---------------|\n")
            rank = 1
            for _, row in df_sorted.iterrows():
                if row['overall'] is not None:
                    f.write(f"| {rank} | {row['LLM']} | {row['overall']:.1f}/10 |\n")
                    rank += 1
            f.write("\n---\n\n")
        
        # Category breakdown (manual markdown table)
        f.write("## 📊 Category Breakdown\n\n")
        
        # Build header
        categories = ['LLM', 'prompt_adherence', 'logo_preservation', 'design_suitability', 
                     'creative_execution', 'technical_quality', 'overall']
        
        # Nice column names
        col_names = {
            'LLM': 'LLM',
            'prompt_adherence': 'Prompt',
            'logo_preservation': 'Logo',
            'design_suitability': 'Design',
            'creative_execution': 'Creative',
            'technical_quality': 'Technical',
            'overall': 'Overall'
        }
        
        # Write header
        header = "| " + " | ".join(col_names[cat] for cat in categories) + " |"
        separator = "|" + "|".join(["------" for _ in categories]) + "|"
        f.write(header + "\n")
        f.write(separator + "\n")
        
        # Write rows
        for _, row in df_data.iterrows():
            values = []
            for cat in categories:
                if cat == 'LLM':
                    values.append(str(row.get(cat, 'N/A')))
                else:
                    val = row.get(cat, None)
                    if val is not None:
                        values.append(f"{val:.1f}")
                    else:
                        values.append("N/A")
            
            f.write("| " + " | ".join(values) + " |\n")
        
        f.write("\n---\n\n")
        
        # Statistics (manual calculation)
        f.write("## 📈 Statistical Summary\n\n")
        
        numeric_cols = ['prompt_adherence', 'logo_preservation', 'design_suitability',
                       'creative_execution', 'technical_quality', 'overall']
        
        f.write("| Category | Mean | Std | Min | Max |\n")
        f.write("|----------|------|-----|-----|-----|\n")
        
        for col in numeric_cols:
            if col in df_data.columns:
                values = df_data[col].dropna()
                if len(values) > 0:
                    col_display = col.replace('_', ' ').title()
                    f.write(f"| {col_display} | {values.mean():.2f} | {values.std():.2f} | "
                           f"{values.min():.1f} | {values.max():.1f} |\n")
        
        f.write("\n")
    
    print(f"📊 Score comparison saved: {report_path}")
    
    # Also save as CSV for easy analysis
    csv_path = output_dir / "scores.csv"
    df_data.to_csv(csv_path, index=False)
    print(f"📊 CSV saved: {csv_path}")

# ===== USAGE =====
analyze_single_prompt_folder(
    "flux_experiments/run_20251116_013857/tshirt_design/prompt_0_base"
)


📁 ANALYZING: prompt_0_base
📝 Prompt: adapt logo for holiday t-shirt print with seasonal elements...
🎨 Scenario: tshirt design

📸 Collecting all images...
   ✅ Found 13 images:
      1. input_image.png
      2. final_output.png
      3. word_attribution_complete.png
      4. ablated_without_elements.png
      5. ablated_without_holiday.png
      6. ablated_without_seasonal.png
      7. ablated_without_t-shirt.png
      8. evolution_grid.png
      9. timestep_evolution_seasonal.png
      10. step_000_t1000.0.png
      11. step_007_t904.5.png
      12. step_014_t759.5.png
      13. step_021_t512.8.png

🤖 claude_3_opus        ❌ Failed (29.5s)
🤖 claude_3_sonnet      ✅ Done (27.6s, 4964 chars)
🤖 claude_3_haiku       ✅ Done (13.6s, 4230 chars)
      📊 Overall Score: 9.0/10
🤖 claude_3.5_sonnet    ✅ Done (21.8s, 3598 chars)
🤖 claude_3.5_haiku     ❌ Failed (1.8s)
🤖 claude_4_opus        ❌ Failed (29.5s)
🤖 claude_4_sonnet      ✅ Done (24.0s, 5967 chars)
🤖 claude_4.5_sonnet    ❌ Failed (29.5s)
🤖 c

({'claude_3_opus': {'llm': 'claude_3_opus',
   'model_id': 'us.anthropic.claude-3-opus-20240229-v1:0',
   'prompt': 'adapt logo for holiday t-shirt print with seasonal elements',
   'scenario': 'tshirt design',
   'analysis': 'ERROR: HTTP 504 - {"message": "Endpoint request timed out"}',
   'scores': None,
   'response_time_seconds': 29.499578952789307,
   'response_length': 59,
   'images_analyzed': 13,
   'timestamp': 1763278067.358587},
  'claude_3_sonnet': {'llm': 'claude_3_sonnet',
   'model_id': 'us.anthropic.claude-3-sonnet-20240229-v1:0',
   'prompt': 'adapt logo for holiday t-shirt print with seasonal elements',
   'scenario': 'tshirt design',
   'analysis': '### 1. Prompt Adherence (3-4 sentences)\nThe final output successfully incorporates seasonal elements like snowflakes, stars, Christmas trees, and poinsettia-like flowers to create a festive holiday design. The original logo is adapted and integrated with these seasonal motifs. However, the prompt specified a "t-shirt pri

In [18]:
import json
from pathlib import Path

# Check error from failed models
failed_models = ['claude_3.5_haiku', 'claude_4_opus']

for model in failed_models:
    json_path = Path(f"flux_experiments/run_20251116_013857/tshirt_design/prompt_0_base/llm_analysis/{model}_analysis.json")
    if json_path.exists():
        with open(json_path) as f:
            data = json.load(f)
            print(f"\n{'='*60}")
            print(f"Model: {model}")
            print(f"{'='*60}")
            print(data['analysis'])


Model: claude_3.5_haiku
ERROR: HTTP 500 - {"error": "Bedrock error: An error occurred (ValidationException) when calling the InvokeModel operation: 'claude-3-5-haiku-20241022' does not support image input."}

Model: claude_4_opus
ERROR: HTTP 504 - {"message": "Endpoint request timed out"}


In [24]:
import requests
import json

API_ENDPOINT = "https://ctwa92wg1b.execute-api.us-east-1.amazonaws.com/prod/invoke"
TEAM_ID = "team_the_great_hack_2025_022"
API_TOKEN = "znqXT5zCmCynAx-kyx_hldrxvSeyaWvxzx55vB5mfNg"

# Try with api_token in the body as well
response = requests.post(
    API_ENDPOINT,
    headers={
        "Content-Type": "application/json",
        "X-Team-ID": TEAM_ID,
        "X-API-Token": API_TOKEN
    },
    json={
        "team_id": TEAM_ID,
        "api_token": API_TOKEN,  # Add this to body
        "model": "us.anthropic.claude-3-5-haiku-20241022-v1:0",
        "messages": [{"role": "user", "content": "Hi"}],
        "max_tokens": 10
    }
)

print(f"Status Code: {response.status_code}")
result = response.json()

if "metadata" in result and "remaining_quota" in result["metadata"]:
    quota = result["metadata"]["remaining_quota"]
    
    print("\n" + "="*50)
    print("HACKATHON BUDGET USAGE")
    print("="*50)
    print(f"Budget Limit:        ${quota['budget_limit']:.2f}")
    print(f"LLM Cost:            ${quota['llm_cost']:.2f}")
    print(f"GPU Cost:            ${quota['gpu_cost']:.2f}")
    print(f"Total Cost:          ${quota['total_cost']:.2f}")
    print(f"Remaining Budget:    ${quota['remaining_budget']:.2f}")
    print(f"Budget Used:         {quota['budget_usage_percent']:.1f}%")
    print("="*50)
else:
    print("\nFull Response:")
    print(json.dumps(result, indent=2))


Status Code: 200

HACKATHON BUDGET USAGE
Budget Limit:        $50.00
LLM Cost:            $4.28
GPU Cost:            $0.00
Total Cost:          $4.28
Remaining Budget:    $45.72
Budget Used:         8.6%


In [8]:
"""
AWS Bedrock API Connection Tester
Tests API connectivity and authentication with different models
"""

import requests
import json
import time

# ===== CONFIGURATION =====
API_ENDPOINT = "https://ctwa92wg1b.execute-api.us-east-1.amazonaws.com/prod/invoke"
TEAM_ID = "team_the_great_hack_2025_022"
API_TOKEN = "znqXT5zCmCynAx-kyx_hldrxvSeyaWvxzx55vB5mfNg"

# ===== TEST 1: Simple Text-Only Request =====
def test_simple_text_request():
    """Test basic API connectivity with text-only prompt"""
    
    print("="*70)
    print("TEST 1: Simple Text Request (No Images)")
    print("="*70)
    
    headers = {
        "Content-Type": "application/json",
        "x-api-key": API_TOKEN
    }
    
    # FIXED: Use snake_case field names
    payload = {
        "team_id": TEAM_ID,  # Changed from teamId
        "api_token": API_TOKEN,  # Added missing field
        "model": "us.anthropic.claude-3-haiku-20240307-v1:0",  # Changed from modelId
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Say 'Hello, API connection working!' in exactly those words."
                    }
                ]
            }
        ],
        "max_tokens": 100,
        "temperature": 0.0
    }
    
    print("\n📤 Sending request to API...")
    print(f"   Endpoint: {API_ENDPOINT}")
    print(f"   Model: Claude Haiku (fastest)")
    
    try:
        start = time.time()
        response = requests.post(API_ENDPOINT, headers=headers, json=payload, timeout=30)
        duration = time.time() - start
        
        print(f"\n📊 Response Status: {response.status_code}")
        print(f"⏱️  Response Time: {duration:.2f}s")
        
        if response.status_code == 200:
            result = response.json()
            print(f"\n✅ SUCCESS!")
            print(f"\n📄 Full Response:")
            print(json.dumps(result, indent=2))
            
            # Extract text if available
            if "content" in result and len(result["content"]) > 0:
                text = result["content"][0].get("text", "No text found")
                print(f"\n💬 LLM Response: {text}")
            
            return True
        else:
            print(f"\n❌ FAILED!")
            print(f"\n📄 Error Response:")
            print(response.text)
            return False
    
    except requests.exceptions.Timeout:
        print(f"\n❌ TIMEOUT (>30s)")
        return False
    
    except Exception as e:
        print(f"\n❌ ERROR: {type(e).__name__}")
        print(f"   Details: {str(e)}")
        return False

# ===== TEST 2: Different Models =====
def test_multiple_models():
    """Test different models to see which ones work"""
    
    print("\n" + "="*70)
    print("TEST 2: Testing Multiple Models")
    print("="*70)
    
    test_models = {
        "Claude Haiku": "us.anthropic.claude-3-haiku-20240307-v1:0",
        "Nova Lite": "us.amazon.nova-lite-v1:0",
        "Llama 11B": "us.meta.llama3-2-11b-instruct-v1:0",
    }
    
    headers = {
        "Content-Type": "application/json",
        "x-api-key": API_TOKEN
    }
    
    results = {}
    
    for model_name, model_id in test_models.items():
        print(f"\n🤖 Testing: {model_name}")
        print(f"   Model ID: {model_id}")
        
        # FIXED: Use snake_case field names
        payload = {
            "team_id": TEAM_ID,
            "api_token": API_TOKEN,
            "model": model_id,
            "messages": [
                {
                    "role": "user",
                    "content": [{"type": "text", "text": "Respond with just 'OK'"}]
                }
            ],
            "max_tokens": 50,
            "temperature": 0.0
        }
        
        try:
            response = requests.post(API_ENDPOINT, headers=headers, json=payload, timeout=20)
            
            if response.status_code == 200:
                result = response.json()
                if "content" in result:
                    text = result["content"][0].get("text", "")
                    print(f"   ✅ Success: {text[:50]}")
                    results[model_name] = "✅ Working"
                else:
                    print(f"   ⚠️  Unexpected response format")
                    results[model_name] = "⚠️ Unexpected format"
            else:
                print(f"   ❌ Failed: {response.status_code}")
                print(f"      {response.text[:200]}")
                results[model_name] = f"❌ Status {response.status_code}"
        
        except Exception as e:
            print(f"   ❌ Error: {str(e)[:100]}")
            results[model_name] = f"❌ {type(e).__name__}"
        
        time.sleep(1)
    
    print("\n" + "="*70)
    print("Model Test Summary:")
    print("="*70)
    for model, status in results.items():
        print(f"  {model:20s} {status}")
    
    return results

# ===== TEST 3: Image Upload Test =====
def test_image_request():
    """Test API with a small test image"""
    
    print("\n" + "="*70)
    print("TEST 3: Image Upload Test")
    print("="*70)
    
    # Create a tiny 1x1 pixel PNG in base64
    tiny_png_base64 = "iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAYAAAAfFcSJAAAADUlEQVR42mNk+M9QDwADhgGAWjR9awAAAABJRU5ErkJggg=="
    
    headers = {
        "Content-Type": "application/json",
        "x-api-key": API_TOKEN
    }
    
    # FIXED: Use snake_case field names
    payload = {
        "team_id": TEAM_ID,
        "api_token": API_TOKEN,
        "model": "us.anthropic.claude-3-haiku-20240307-v1:0",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/png",
                            "data": tiny_png_base64
                        }
                    },
                    {
                        "type": "text",
                        "text": "Describe this image in one word."
                    }
                ]
            }
        ],
        "max_tokens": 50,
        "temperature": 0.0
    }
    
    print("\n📤 Sending 1x1 pixel test image...")
    
    try:
        response = requests.post(API_ENDPOINT, headers=headers, json=payload, timeout=30)
        
        print(f"📊 Status: {response.status_code}")
        
        if response.status_code == 200:
            result = response.json()
            print(f"✅ Image request successful!")
            print(f"\n📄 Response:")
            print(json.dumps(result, indent=2))
            return True
        else:
            print(f"❌ Failed!")
            print(f"\n📄 Error:")
            print(response.text)
            return False
    
    except Exception as e:
        print(f"❌ ERROR: {str(e)}")
        return False

# ===== TEST 4: Check Response Format =====
def test_response_format():
    """Check the exact response structure from API"""
    
    print("\n" + "="*70)
    print("TEST 4: Response Format Analysis")
    print("="*70)
    
    headers = {
        "Content-Type": "application/json",
        "x-api-key": API_TOKEN
    }
    
    # FIXED: Use snake_case field names
    payload = {
        "team_id": TEAM_ID,
        "api_token": API_TOKEN,
        "model": "us.anthropic.claude-3-haiku-20240307-v1:0",
        "messages": [
            {
                "role": "user",
                "content": [{"type": "text", "text": "What is 2+2?"}]
            }
        ],
        "max_tokens": 50
    }
    
    try:
        response = requests.post(API_ENDPOINT, headers=headers, json=payload, timeout=20)
        
        print(f"\nStatus Code: {response.status_code}")
        print(f"Headers: {dict(response.headers)}")
        
        if response.status_code == 200:
            result = response.json()
            
            print(f"\n📊 Response Structure:")
            print(f"   Type: {type(result)}")
            print(f"   Keys: {list(result.keys())}")
            
            print(f"\n📄 Full JSON:")
            print(json.dumps(result, indent=2))
            
            # Try different extraction methods
            print(f"\n🔍 Extraction Attempts:")
            
            if "content" in result:
                print(f"   ✅ Has 'content' key")
                if len(result["content"]) > 0:
                    print(f"   ✅ Content has items: {len(result['content'])}")
                    if "text" in result["content"][0]:
                        print(f"   ✅ First item has 'text'")
                        print(f"   📝 Text: {result['content'][0]['text']}")
            
            if "completion" in result:
                print(f"   ✅ Has 'completion' key: {result['completion']}")
            
            if "message" in result:
                print(f"   ✅ Has 'message' key: {result['message']}")
            
            return True
        else:
            print(f"\n❌ Error Response:")
            print(response.text)
            return False
    
    except Exception as e:
        print(f"\n❌ Exception: {str(e)}")
        import traceback
        traceback.print_exc()
        return False

# ===== RUN ALL TESTS =====
def run_all_tests():
    """Run complete diagnostic suite"""
    
    print("\n" + "🔬"*35)
    print("AWS BEDROCK API DIAGNOSTIC SUITE")
    print("🔬"*35 + "\n")
    
    # Run tests
    test1 = test_simple_text_request()
    time.sleep(2)
    
    test2 = test_multiple_models()
    time.sleep(2)
    
    test3 = test_image_request()
    time.sleep(2)
    
    test4 = test_response_format()
    
    # Summary
    print("\n" + "="*70)
    print("DIAGNOSTIC SUMMARY")
    print("="*70)
    print(f"  Test 1 (Simple Text):   {'✅ PASS' if test1 else '❌ FAIL'}")
    print(f"  Test 2 (Multi-Model):   {'✅ PASS' if any('✅' in v for v in test2.values()) else '❌ FAIL'}")
    print(f"  Test 3 (Image Upload):  {'✅ PASS' if test3 else '❌ FAIL'}")
    print(f"  Test 4 (Format Check):  {'✅ PASS' if test4 else '❌ FAIL'}")
    print("="*70)
    
    if not test1:
        print("\n⚠️  DIAGNOSIS: Basic API connection failing")
        print("   Check:")
        print("   1. API endpoint URL is correct")
        print("   2. API token is valid")
        print("   3. Team ID is correct")
        print("   4. Network/firewall allows connection")
    
    elif test1 and not test3:
        print("\n⚠️  DIAGNOSIS: Text works, images fail")
        print("   Check:")
        print("   1. Image encoding (base64 format)")
        print("   2. Model supports vision (Claude 3+, Nova, Pixtral)")
        print("   3. Image size limits")

# ===== EXECUTE =====
run_all_tests()


🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬
AWS BEDROCK API DIAGNOSTIC SUITE
🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬

TEST 1: Simple Text Request (No Images)

📤 Sending request to API...
   Endpoint: https://ctwa92wg1b.execute-api.us-east-1.amazonaws.com/prod/invoke
   Model: Claude Haiku (fastest)

📊 Response Status: 200
⏱️  Response Time: 0.36s

✅ SUCCESS!

📄 Full Response:
{
  "content": [
    {
      "type": "text",
      "text": "Hello, API connection working!"
    }
  ],
  "usage": {
    "input_tokens": 21,
    "output_tokens": 9,
    "total_tokens": 30
  },
  "metadata": {
    "team_id": "team_the_great_hack_2025_022",
    "model": "us.anthropic.claude-3-haiku-20240307-v1:0",
    "cost_usd": 1.7e-05,
    "latency_ms": 254.89,
    "remaining_quota": {
      "requests_today": 1,
      "tokens_today": 30,
      "llm_cost": 2.1521115,
      "gpu_cost": 0.0,
      "total_cost": 2.1521115,
      "budget_limit": 50.0,
      "remaining_budget": 47.847888499999996,
      "budget_usage_percent": 4.3